In [1]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
import math
from numpy import random
from IPython.display import HTML
import torchvision.models as torch_models
from base64 import b64encode
import os
from IPython.display import Video
from utils import *
from torchvision import transforms
from PIL import Image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else device
print(device)
!nvidia-smi

mps
zsh:1: command not found: nvidia-smi


In [17]:
yolo_path = "best.pt"
model = YOLO(yolo_path)
classNames = ['basketball', 'hoop', 'person']


videos = [
    "/Users/oscarwan/bballDetection/videos/testing_videos/20230705_Game1.mp4",
    # "videos/2023_11_15_id_OR9LPl8QDc.mp4",
    #"videos/2023_11_15_id_jM9CfUbroAg.mp4", 
    #"videos/2023_11_15_id_tPesRuo9YxAt53s.mp4",
]


stamps = []
for video_path in videos:
    # video_path = "videos/" + video_path
    video_name = video_path.split("/")[-1]
    #output_path = "videos/inferenced_" + video_name.replace(".webm", ".mp4")
    output_path = "output.mp4"
    compressed_output_path = "videos/compressed_inferenced_" + video_name.replace(".webm", ".mp4")
    if os.path.exists(output_path) or os.path.exists(compressed_output_path):
        if os.path.exists(compressed_output_path):
            continue
        else:
            subprocess.run(['ffmpeg', '-y', '-hide_banner',  '-loglevel error', '-i', output_path, '-vcodec', 'libx264', compressed_output_path], check=False)
            continue
    
    cap, fps, frame_width, frame_height = get_video_info(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width,frame_height))
    if video_name.endswith(".webm"):
        codec = cv2.VideoWriter_fourcc(*'mp4v')
    else:
        codec = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, codec, fps, (frame_width,frame_height))
    batch_size = 128 * 2
    num_batches = math.ceil(total_frames / batch_size)

    results = []

    score_timestamps = []
    
    count = 0
    score = 0
    display_prob = [0.0]
    for i in tqdm(range(num_batches)):
        frame_count = 0
        frames = []
        for i in range(batch_size):
            ret, img = cap.read()
            if ret:
                frames.append(img)
            else:
                break

        results = model(frames, stream=False, verbose = False, batch = len(frames))
        cropped_images_batch = []
        box_info = []  # To store additional info like coordinates
        for c, r in enumerate(results):
            img = r.orig_img
            boxes = r.boxes
            cropped_images = []
            count += 1
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # convert to int values
                if model.names[int(box.cls)] == "hoop":
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img, f"{classNames[int(box.cls)]}: {box.conf:.3f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    
                    # if x1 > x2 or y1 > y2:
                    #     continue
                    # else:
                        # cropped_img = img[y1:y2, x1:x2]
                        # cropped_images.append(cropped_img)
                        # cropped_img, pred, prob = predict_hoop_box(img, cls_model, x1, y1, x2, y2, preprocess, device, threshold=0.7)
                        # if pred == 1:
                        #     score += 1
                        #     count = 0
                        #     current_frame = i * batch_size + c
                        #     time_stamp = current_frame / fps
                        #     score_timestamps.append(time_stamp)
                        #     cropped_img.save(f"cropped/{round(time.time())}_{round(prob * 100)}_{score}.jpg")
                        
                elif model.names[int(box.cls)] == "basketball":
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, f"{classNames[int(box.cls)]}: {box.conf:.3f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                elif model.names[int(box.cls)] == "person":
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(img, f"{classNames[int(box.cls)]}: {box.conf:.3f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # if len(cropped_images) == 0:
            #     continue
            # pred, prob = prepare_hoop_box_batch(cropped_images, cls_model,  preprocess, device, threshold=0.6)
            # if pred.sum() > 0 and count > 20:
            #     score += 1
            #     count = 0
            #     current_frame = i * batch_size + c
            #     time_stamp = current_frame / fps
            #     score_timestamps.append(time_stamp)
            #     display_prob = prob
        
            # cv2.putText(img, f'Score: {score}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
            # cv2.putText(img, f'Prob: {max(display_prob):.3f}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
            out.write(img)
            cv2.imshow("img", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        if not ret:
            break
        
    out.release()
    cap.release()
    


Unrecognized option 'loglevel error'.
Error splitting the argument list: Option not found


In [15]:
device

device(type='mps')

In [18]:
video_path = "videos_full/id_dF2eSE2FQ4.mp4"
cap = cv2.VideoCapture(video_path)
while True:
    ret, img = cap.read()
    if ret:
        results = model(img, device = "mps")
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                class_name = model.names[int(box.cls)]
                conf = box.conf.item()
                if class_name == "hoop":
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img, f"{class_name}: {conf:.3f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                elif class_name == "basketball":
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, f"{class_name}: {conf:.3f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                elif class_name == "person":
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(img, f"{class_name}: {conf:.3f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow("img", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
                
    else:
        break
    


0: 416x736 1 hoop, 7 persons, 313.1ms
Speed: 13.4ms preprocess, 313.1ms inference, 29.7ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 5 persons, 55.3ms
Speed: 4.8ms preprocess, 55.3ms inference, 25.6ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 6 persons, 43.7ms
Speed: 2.6ms preprocess, 43.7ms inference, 24.8ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 6 persons, 42.9ms
Speed: 2.9ms preprocess, 42.9ms inference, 11.2ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 6 persons, 37.2ms
Speed: 3.1ms preprocess, 37.2ms inference, 11.6ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 6 persons, 38.1ms
Speed: 2.8ms preprocess, 38.1ms inference, 11.4ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 7 persons, 37.9ms
Speed: 2.5ms preprocess, 37.9ms inference, 11.3ms postprocess per image at shape (1, 3, 416, 736)

0: 416x736 1 hoop, 8 persons, 38.1ms
Speed: 

In [19]:
cv2.destroyAllWindows()

In [7]:
detect_video(model, video_path)


0: 416x736 4 basketballs, 3 hoops, 267.4ms
Speed: 2.2ms preprocess, 267.4ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 736)


TypeError: unsupported format string passed to Tensor.__format__

In [9]:
display_compressed_video(output_path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab